In [ ]:
# Projeto 13 – Previsão de Pedidos de Táxi por Hora com Séries Temporais

# 1. Bibliotecas
# Importamos as bibliotecas necessárias para análise de dados, visualização e machine learning
import pandas as pd  # manipulação de dados
import numpy as np  # operações matemáticas e arrays
import matplotlib.pyplot as plt  # visualização gráfica
import seaborn as sns  # visualização mais estilizada
from sklearn.linear_model import LinearRegression  # modelo de regressão linear
from sklearn.ensemble import RandomForestRegressor  # modelo Random Forest
from sklearn.metrics import mean_squared_error  # métrica de erro RMSE
from sklearn.model_selection import train_test_split  # separação dos dados em treino e teste

# 2. Carregar os dados
# Carregamos o arquivo CSV e configuramos a coluna datetime como índice com tipo de data
df = pd.read_csv('/mnt/data/taxi.csv', parse_dates=['datetime'], index_col='datetime')

# 3. Reamostragem para 1 hora
# Os dados estão em intervalos de 10 minutos, aqui somamos os pedidos para intervalos de 1 hora
df = df.resample('1H').sum()

# 4. Análise Exploratória
# Visualizamos os pedidos de táxi ao longo do tempo para entender padrões e sazonalidades
plt.figure(figsize=(12, 4))
df['num_orders'].plot(title='Pedidos de Táxi por Hora')
plt.ylabel('nº Pedidos')
plt.grid()
plt.show()

# 5. Engenharia de Atributos
# Criamos novas colunas (features) para ajudar o modelo a aprender padrões dos dados

def make_features(data, max_lag, rolling_mean_size):
    data['hour'] = data.index.hour  # extrai a hora do dia
    data['dayofweek'] = data.index.dayofweek  # extrai o dia da semana
    for lag in range(1, max_lag + 1):
        data[f'lag_{lag}'] = data['num_orders'].shift(lag)  # cria colunas defasadas (lags)
    data['rolling_mean'] = data['num_orders'].shift().rolling(rolling_mean_size).mean()  # média móvel
    return data

make_features(df, max_lag=6, rolling_mean_size=3)
df = df.dropna()  # remove valores nulos gerados pelas defasagens e médias

# 6. Separar dados
# Separamos os dados em variáveis independentes (X) e alvo (y)
features = df.drop('num_orders', axis=1)
target = df['num_orders']

# Dividimos os dados em treino + validação e teste (90% treino + val / 10% teste)
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, shuffle=False)

# Dividimos o treino + validação em treino e validação (80% treino / 20% validação)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=False)

# 7. Modelos
## Regressão Linear
# Treinamos o modelo de regressão linear com os dados de treino
lr = LinearRegression()
lr.fit(X_train, y_train)

# Fazemos previsões com o modelo treinado nos dados de validação
val_preds_lr = lr.predict(X_valid)

# Calculamos o erro das previsões usando RMSE (raiz do erro quadrático médio)
rmse_lr = np.sqrt(mean_squared_error(y_valid, val_preds_lr))
print(f"Regressão Linear - RMSE validação: {rmse_lr:.2f}")

## Random Forest
# Treinamos o modelo de Random Forest (100 árvores)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Previsões na validação
val_preds_rf = rf.predict(X_valid)

# Cálculo do RMSE para o modelo Random Forest
rmse_rf = np.sqrt(mean_squared_error(y_valid, val_preds_rf))
print(f"Random Forest - RMSE validação: {rmse_rf:.2f}")

# 8. Avaliação Final com Teste
# Escolhemos o modelo com menor erro para aplicar no conjunto de teste
final_model = rf if rmse_rf < rmse_lr else lr

# Fazemos previsões no conjunto de teste com o modelo escolhido
test_preds = final_model.predict(X_test)

# Calculamos o RMSE final no conjunto de teste
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"\nMelhor modelo: {type(final_model).__name__}")
print(f"RMSE no conjunto de teste: {rmse_test:.2f}")

# 9. Conclusão
# Avaliamos se o modelo atinge a meta de RMSE <= 48
if rmse_test <= 48:
    print("\nMeta alcançada: RMSE ≤ 48 ✅")
else:
    print("\nMeta não alcançada: RMSE > 48 ❌")

# Explicação Final:
# Neste projeto, preparamos os dados com reamostragem e engenharia de atributos temporais.
# Testamos dois modelos: Regressão Linear (simples) e Random Forest (mais robusto).
# Avaliamos os modelos com a métrica RMSE para garantir previsões precisas.
# O modelo final escolhido foi aquele com melhor desempenho na validação e teste.
# O objetivo do projeto era prever com exatidão o número de pedidos de táxi para a próxima hora,
# auxiliando a empresa a tomar decisões sobre a alocação de motoristas nos aeroportos.
